In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqn
%matplotlib inline
import sys
sys.path.append('..')
from meters.batch import MeterBatch
from meters.pipelines import PipelineFactory
from meters.dataset import Dataset, FilesIndex
import meters as ms
from meters.batch import MeterBatch
from meters.dataset import B,V,F, FilesIndex, Dataset, Pipeline
from meters.dataset.dataset.models.tf import ResNet101, DenseNet121, VGG19, MobileNet_v2, TFModel

# train on digits

In [30]:
src = './../data/outcomes_data/images/'
fileindex = FilesIndex(path=src + '*.png', no_ext=True)
dset_digits = Dataset(fileindex, batch_class=MeterBatch)
dset_digits.cv_split()

In [31]:
pf = PipelineFactory()
train_ppl = pf.train_to_digits(src='./../data/outcomes_data', 
                               shape=(64,32),
                               ppl_config={'model': VGG19,
                                        'model_name': 'digits',
                                        'batch_size': 10,
                                        'n_epochs': None,
                                        'save': True})

In [32]:
ppl_new = dset_digits.train>>train_ppl
for i in tqn(range(100)):
    batch = ppl_new.next_batch()
plt.plot(ppl_new.get_variable('current_loss'))

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


ValueError: setting an array element with a sequence.

# pred on digits

In [14]:
load_pred_ppl = pf.predict_digits(shape=(64, 32), src='./../data/outcomes_data/', ppl_config={'model_name':'digits',
                                                                                              'model':'./digits/'})

In [8]:
config = {'model_name': 'digits',
         'model': ppl_new}
pred_ppl = pf.predict_digits(shape=(64, 32), src='./../data/outcomes_data/', ppl_config=config)

In [16]:
ppl_new = dset_digits.train>>load_pred_ppl
batch = ppl_new.next_batch()

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


INFO:tensorflow:Restoring parameters from ./digits/model-0


INFO:tensorflow:Restoring parameters from ./digits/model-0


In [17]:
ppl_new.get_variable('digits_prediction'), batch.labels

([array([[ 5.65748962e-07,  3.52259093e-08,  1.25158795e-06,
           2.14221359e-07, -5.89833746e-07,  1.95256277e-07,
          -6.93611469e-07, -3.20761330e-07, -5.83061365e-07,
          -2.41722046e-06],
         [ 5.97423366e-07,  4.61575915e-08,  1.35837035e-06,
           2.50969975e-07, -6.17386888e-07,  1.77442260e-07,
          -7.74470152e-07, -4.02934461e-07, -5.79477216e-07,
          -2.60640627e-06],
         [ 5.35198467e-07, -3.72890980e-08,  1.35227765e-06,
           2.53926942e-07, -6.48857906e-07,  1.97549824e-07,
          -8.51262257e-07, -4.09658526e-07, -7.18714773e-07,
          -2.60139836e-06],
         [ 6.34758351e-07,  9.69362457e-09,  1.43188265e-06,
           1.81012197e-07, -6.43524174e-07,  2.44806870e-07,
          -8.60642388e-07, -4.87808393e-07, -7.45037994e-07,
          -2.83497388e-06],
         [ 6.44861984e-07, -6.50377743e-08,  1.66880147e-06,
           3.01153307e-07, -7.06780725e-07,  2.36307642e-07,
          -1.07745950e-06, -3.9641

# simple train bb

In [18]:
src = './../data/outcomes_data/images/'
fileindex = FilesIndex(path=src + '*.png', no_ext=True)
dset = Dataset(fileindex, batch_class=MeterBatch)
dset.cv_split()

pf = PipelineFactory()
src = '.'
batch_size = 10

config = {'model': MobileNet_v2,
          'model_name': 'model',
          'batch_size': batch_size, 
}

import tensorflow as tf
session_config = tf.ConfigProto()
session_config.gpu_options.allow_growth = True

model_config = {
     'session' : {'config' : session_config},
     'body': {'width_factor' : 1},
     'inputs' : {'images': {'shape': B('image_shape'),
                            'data_format': 'channels_last'},
                 'labels': {'name': 'targets', 'shape' : (4,)}},
     'loss' : {'name' : 'huber', 'delta' : 1},
     'decay' : {'name':"exp", "decay_rate" : 0.95, 'learning_rate' : 1.,\
                "decay_steps" : 10, 'staircase' : True},
     'optimizer': {'name' : 'Momentum', 'momentum' : 0.99, 'use_nesterov' : True},
     'input_block/inputs' : 'images'}

In [19]:
ppl = pf.train_to_coordinates('./../data/outcomes_data/', ppl_config=config, model_config=model_config)

In [20]:
ppl_new = dset.train>>ppl
for i in tqn(range(100)):
    batch = ppl_new.next_batch()
plt.plot(ppl_new.get_variable('current_loss'))

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


KeyboardInterrupt: 

# simple pred bb

In [21]:
tf.reset_default_graph()

In [23]:
config.update(model_name='model', model='./mobilenet_v2/')
ppl = pf.predict_coordinates(src='./../data/augment_data/', ppl_config=config)

In [24]:
ppl_pred_new = dset.train>>ppl
for i in tqn(range(1)):
    batch = ppl_pred_new.next_batch()
#     print(batch.labels)
#     plt.imshow(batch.images[0])
#     plt.show()

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "



INFO:tensorflow:Restoring parameters from ./mobilenet_v2/model-45950


INFO:tensorflow:Restoring parameters from ./mobilenet_v2/model-45950


In [25]:
ppl_pred_new.get_variable('prediction')

[array([[11, 17, 56, 20],
        [17, 25, 76, 15],
        [12, 16, 50, 17],
        [33, 45, 53, 12],
        [10, 49, 79, 16],
        [24, 40, 62, 16],
        [31, 48, 50, 12],
        [ 8, 43, 75, 15],
        [20, 13, 72, 18],
        [23, 32, 68, 18]], dtype=int32)]

# pred digits on pred bb

In [33]:
ppl = pf.simple_predict_numbers('model', './mobilenet_v2/', 'digits', ppl_new, src='./../data/outcomes_data/',
                                ppl_config_bb={}, ppl_config_digits = {})

In [34]:
ppl_new = dset.train>>ppl
batch = ppl_new.next_batch()

/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


INFO:tensorflow:Restoring parameters from ./mobilenet_v2/model-45950


INFO:tensorflow:Restoring parameters from ./mobilenet_v2/model-45950


In [35]:
ppl_new.get_variable('digits_prediction')

[array([[ 21.442673  ,   0.98112625,  -6.5644264 ,  20.679684  ,
         -49.891125  ,  21.876451  ,  19.530243  , -29.76316   ,
          21.069157  ,   6.2498465 ],
        [ 21.44262   ,   0.9810794 ,  -6.564439  ,  20.679659  ,
         -49.89098   ,  21.876392  ,  19.530205  , -29.76304   ,
          21.069117  ,   6.249802  ],
        [ 21.442633  ,   0.9810844 ,  -6.564433  ,  20.679659  ,
         -49.891     ,  21.876398  ,  19.530205  , -29.763042  ,
          21.06912   ,   6.249811  ],
        [ 21.442587  ,   0.98102957,  -6.5644403 ,  20.679623  ,
         -49.890835  ,  21.876331  ,  19.530142  , -29.762922  ,
          21.069084  ,   6.2497663 ],
        [ 21.442606  ,   0.9810521 ,  -6.5644374 ,  20.679625  ,
         -49.890907  ,  21.876362  ,  19.53018   , -29.762972  ,
          21.069098  ,   6.2497873 ],
        [ 21.442612  ,   0.98105866,  -6.5644393 ,  20.679626  ,
         -49.890923  ,  21.87637   ,  19.530178  , -29.762989  ,
          21.06911   ,   6.249